In [2]:
import pygame

pygame.init()
#Font
Font = pygame.font.SysFont('Arial',35)

pygame 2.0.1 (SDL 2.0.14, Python 3.6.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:

#Color for drawing
black = (0, 0, 0)
white = (255, 255, 255)
red = (255, 0, 0)
blue = (0,0,255)


big_cell = 3 #number of big cells that we want. Works well from 2-6, after that the ratio looks weird.
cell_r = 75 #cell radius
distance_between_cell = 25 #Distance between cells
distance_between_radius = cell_r * 2 + distance_between_cell
next_game = big_cell+1

position = [] #Position on the Board
blue_possible = [] #Auxilary list to represent player 2 move on the small board
red_possible = [] #Auxilary list to represent player 1 move on the small board
Win_con_op = [] #Auxilary list for the big board
Big_board_position = [] #Position on the big board. 
Big_board_possible_red = []
Big_board_possible_blue = []
ref_N = []

for row in range(big_cell):
    position.append([])
    blue_possible.append([])
    red_possible.append([])
    Win_con_op.append(0)
    Big_board_position.append(0)
    Big_board_possible_red.append('Y')
    Big_board_possible_blue.append('Y')
    ref_N.append('N')
    for column in range(big_cell):
        position[row].append(0)
        blue_possible[row].append('Y')
        red_possible[row].append('Y')



window_size = [(big_cell * cell_r * 2) + ((big_cell) * distance_between_cell) + 5, cell_r*3]
width = 330 - (window_size[0] // 2)
height = 330 - (window_size[1] // 2)

size = 660,660
screen = pygame.display.set_mode(size)
pygame.display.set_caption("SNORT on a Path")


board_img = pygame.Surface(window_size,pygame.SRCALPHA)
small_board_move = pygame.Surface(window_size,pygame.SRCALPHA)
big_board_move = pygame.Surface(window_size,pygame.SRCALPHA)
blinking = pygame.Surface(window_size,pygame.SRCALPHA)
end_screen_base = pygame.Surface(window_size,pygame.SRCALPHA)
button_screen = pygame.Surface((size),pygame.SRCALPHA)
board_img.fill(white)

#Start drawing the board
Big_circle_center = []
Small_circle_center = []
for circle in range(big_cell): #Outside game
    center_big = ((cell_r+distance_between_cell/2)+(circle*distance_between_radius),cell_r + cell_r/2)
    Big_circle_center.append(center_big)
    pygame.draw.circle(board_img,black,center_big, cell_r,width=2)
    start = (cell_r+distance_between_cell/2)+(circle*distance_between_radius) - cell_r 
    end = (cell_r+distance_between_cell/2)+(circle*distance_between_radius) + cell_r
    interval = (end-start) / big_cell
    small_r = (cell_r / big_cell) / 1.5
    small_distance_between_radius = small_r + interval 
    Small_circle_center.append([])
    pygame.draw.line(board_img,black,(interval + (circle * distance_between_radius),cell_r + cell_r/2),((big_cell-((big_cell-1)*0.5))*interval + big_cell*small_r + (circle * distance_between_radius),cell_r + cell_r/2))
    for circle_small in range(big_cell): #Inside game board
        center_small = ((small_r + interval/2) + (circle_small * interval) + (circle * distance_between_radius), cell_r + cell_r/2)
        Small_circle_center[circle].append(center_small)
        pygame.draw.circle(board_img,black,center_small,small_r,width=3)
        pygame.draw.circle(board_img,white,center_small,small_r-3,width=0)

for circle in range(big_cell-1): #Connect line for outside game
    pygame.draw.line(board_img,black,(2*cell_r + circle*distance_between_radius + distance_between_cell/2,cell_r + cell_r/2), (2*cell_r + circle*distance_between_radius + 1.5*distance_between_cell,cell_r + cell_r/2))

#End drawing the board

#Start gameover screen
end_screen_base.fill(white)
P1_Win = Font.render('Player 1 Wins!' , True , red)
P1_Win_size = Font.size('Player 1 Wins!')
P2_Win = Font.render('Player 2 Wins' , True , blue)
P2_Win_size = Font.size('Player 2 Wins!')
#End gameover screen

clock = pygame.time.Clock()

count = [] #We use this list to save the number of move when a board finishes 
k = 0
c = '0'
for i in range(big_cell):
    count.append(0)

alpha = 1
change = True
done = False
blink_spd = 0.15
pos = (0,0)


while not done:      
    #Mouseclick event
    for event in pygame.event.get(): 
        if event.type == pygame.QUIT: 
            done = True 
        elif event.type == pygame.MOUSEBUTTONDOWN:
            pos = pygame.mouse.get_pos()
            
            
    #Blinking effect      
    if change == True:
        alpha = alpha + blink_spd
        if alpha >= 175:
            change = False
    elif change == False:
        alpha = alpha - blink_spd
        if alpha <= 30:
            change = True
            
    if ((all(i == 'N' for i in Big_board_possible_red) == True and Winner == 0)
        or ((all(i == ref_N for i in red_possible)) and k % 2 == 0 and Winner == 0 )):
            Winner = 2
    elif ((all(i == 'N' for i in Big_board_possible_blue) == True and Winner == 0)
        or ((all(i == ref_N for i in blue_possible)) and k % 2 == 1 and Winner == 0)):
            Winner = 1
            
            
    #Move set instructions
    for i in range(len(Small_circle_center)):
        # Draw for the big board
        if Big_board_position[i] == 1:
            pygame.draw.circle(big_board_move,red,Big_circle_center[i],cell_r,width=3)
        elif Big_board_position[i] == 2:
            pygame.draw.circle(big_board_move,blue,Big_circle_center[i],cell_r,width=3)
        #Draw the blinking effect for big board
        elif Win_con_op[i] == 1 and Big_board_possible_red[i] == 'Y' and Big_board_position[i] == 0 and k % 2 == 0:
            pygame.draw.circle(blinking,red,Big_circle_center[i],cell_r+3,width=3)
        elif Win_con_op[i] == 2 and Big_board_possible_blue[i] == 'Y' and Big_board_position[i] == 0 and k % 2 == 1:
            pygame.draw.circle(blinking,blue,Big_circle_center[i],cell_r+3,width=3)
        #Save the current value of k when a board is finish and record it 
        elif (all(j == 'N' for j in red_possible[i]) == True):
            next_game = big_cell+1 #If there is no possible move on a board i, then the player free to move anywhere
            if Win_con_op[i] == 0:
                Win_con_op[i] = 2 # small boards winning conditions checks
        elif (all(j == 'N' for j in blue_possible[i]) == True):
            next_game = big_cell+1
            if Win_con_op[i] == 0:
                Win_con_op[i] = 1 # small boards winning conditions checks
        else:
            pygame.draw.circle(blinking,white,Big_circle_center[i],cell_r+3,width=3)
            
            
        for j in range(len(Small_circle_center[i])):
            #Start outer board move set instruction
            if (Win_con_op[i] == 1 and k % 2 == 0
                and (width + Big_circle_center[i][0] - cell_r) < pos[0] 
                and pos[0] < (width + Big_circle_center[i][0] + cell_r)
                and (height + Big_circle_center[i][1] - cell_r) < pos[1] 
                and pos[1] < (height + Big_circle_center[i][1] + cell_r)
                and ((width + Small_circle_center[i][j][0] - small_r) > pos[0]  
                or pos[0] > (width + Small_circle_center[i][j][0] + small_r)) 
                and ((height + Small_circle_center[i][j][1] - small_r) > pos[1] 
                or pos[1] > (height + Small_circle_center[i][j][1] + small_r)) 
               ):
                if i == 0:
                    if Big_board_position[i+1] == 2:
                        Big_board_position[i] = 0
                    else:
                        Big_board_position[i] = 1
                        next_game = big_cell+1
                        k = k + 1 
                        Big_board_possible_red[i] = 'N'
                        Big_board_possible_blue[i] = 'N'
                        Big_board_possible_blue[i+1] = 'N'
                elif i == (len(Big_board_possible_red) - 1):
                    if Big_board_position[i-1] == 2:
                        Big_board_position[i] = 0
                    else:
                        Big_board_position[i] = 1
                        next_game = big_cell+1
                        k = k + 1
                        Big_board_possible_red[i] = 'N'
                        Big_board_possible_blue[i] = 'N'
                        Big_board_possible_blue[i-1] = 'N'
                else:
                    if Big_board_position[i+1] == 2 or Big_board_position[i-1] == 2:
                        Big_board_position[i] = 0
                    else:
                        Big_board_position[i] = 1
                        next_game = big_cell+1
                        k = k + 1
                        Big_board_possible_red[i] = 'N'
                        Big_board_possible_blue[i] = 'N'
                        Big_board_possible_blue[i+1] = 'N'
                        Big_board_possible_blue[i-1] = 'N'
            elif (Win_con_op[i] == 2 and k % 2 == 1
                and (width + Big_circle_center[i][0] - cell_r) < pos[0]
                and pos[0] < (width + Big_circle_center[i][0] + cell_r)
                and (height + Big_circle_center[i][1] - cell_r) < pos[1]
                and pos[1] < (height + Big_circle_center[i][1] + cell_r)
                and ((width + Small_circle_center[i][j][0] - small_r) > pos[0]  
                or pos[0] > (width + Small_circle_center[i][j][0] + small_r)) 
                and ((height + Small_circle_center[i][j][1] - small_r) > pos[1] 
                or pos[1] > (height + Small_circle_center[i][j][1] + small_r)) 
               ):
                if i == 0:
                    if Big_board_position[i+1] == 1:
                        Big_board_position[i] = 0
                    else:
                        Big_board_position[i] = 2
                        next_game = big_cell+1
                        k = k + 1
                        Big_board_possible_blue[i] = 'N'
                        Big_board_possible_red[i] = 'N'
                        Big_board_possible_red[i+1] = 'N'
                elif i == (len(Big_board_possible_red) - 1):
                    if Big_board_position[i-1] == 1:
                        Big_board_position[i] = 0
                    else:
                        Big_board_position[i] = 2
                        next_game = big_cell+1
                        k = k + 1
                        Big_board_possible_blue[i] = 'N'
                        Big_board_possible_red[i] = 'N'
                        Big_board_possible_red[i-1] = 'N'
                else:
                    if Big_board_position[i+1] == 1 or Big_board_position[i-1] == 1:
                        Big_board_position[i] = 0
                    else:
                        Big_board_position[i] = 2
                        next_game = big_cell+1
                        k = k + 1
                        Big_board_possible_blue[i] = 'N'
                        Big_board_possible_red[i] = 'N'
                        Big_board_possible_red[i+1] = 'N'
                        Big_board_possible_red[i-1] = 'N'
            #End outer board move set instruction

            #Start inner boards move set instruction
            elif ((width + Small_circle_center[i][j][0] - small_r) < pos[0] 
                and pos[0] < (width + Small_circle_center[i][j][0] + small_r) 
                and (height + Small_circle_center[i][j][1] - small_r) < pos[1] 
                and pos[1] < (height + Small_circle_center[i][j][1] + small_r) 
                and k % 2 == 0
               ): # Player 1 select a circle
                if position[i][j] == 0 and (next_game == i or next_game == big_cell+1):
                    if j == 0: #Left edge
                        if position[i][j+1] == 2: 
                            position[i][j] = 0 
                        else:
                            position[i][j] = 1
                            red_possible[i][j] = 'N'
                            blue_possible[i][j] = 'N'
                            blue_possible[i][j+1] = 'N'
                            next_game = j
                            k = k + 1
                    elif j == (len(Small_circle_center[i]) - 1): #Right edge
                        if position[i][j-1] == 2:
                            position[i][j] = 0
                        else:
                            position[i][j] = 1
                            next_game = j
                            red_possible[i][j] = 'N'
                            blue_possible[i][j] = 'N'
                            blue_possible[i][j-1] = 'N'
                            k = k + 1
                    else: #Normal
                        if position[i][j-1] == 2 or position[i][j+1] == 2:
                            position[i][j] = 0
                        else:
                            position[i][j] = 1
                            next_game = j
                            red_possible[i][j] = 'N'
                            blue_possible[i][j] = 'N'
                            blue_possible[i][j-1] = 'N'
                            blue_possible[i][j+1] = 'N'
                            k = k + 1                    
            elif (((width + Small_circle_center[i][j][0] - small_r) < pos[0] 
                and pos[0] < (width + Small_circle_center[i][j][0] + small_r)) 
                and ((height + Small_circle_center[i][j][1] - small_r) < pos[1] 
                and pos[1] < (height + Small_circle_center[i][j][1] + small_r))
                and k % 2 == 1
                 ): # Player 2 select a circle
                if position[i][j] == 0 and (next_game == i or next_game == big_cell+1):
                    if j == 0: #Left edge
                        if position[i][j+1] == 1: 
                            position[i][j] = 0 
                        else:
                            position[i][j] = 2
                            next_game = j
                            blue_possible[i][j] = 'N'
                            red_possible[i][j] = 'N'
                            red_possible[i][j+1] = 'N'
                            k = k + 1
                    elif j == (len(Small_circle_center[i]) - 1): #Right edge
                        if position[i][j-1] == 1:
                            position[i][j] = 0
                        else:
                            position[i][j] = 2
                            next_game = j
                            blue_possible[i][j] = 'N'
                            red_possible[i][j] = 'N'
                            red_possible[i][j-1] = 'N'
                            k = k + 1
                    else: #Normal
                        if position[i][j-1] == 1 or position[i][j+1] == 1:
                            position[i][j] = 0
                        else:
                            position[i][j] = 2
                            next_game = j
                            blue_possible[i][j] = 'N'
                            red_possible[i][j] = 'N'
                            red_possible[i][j-1] = 'N'
                            red_possible[i][j+1] = 'N'
                            k = k + 1 
        #End inner boards move set instruction
            #Draw on the small board
            elif position[i][j] == 1:
                pygame.draw.circle(small_board_move,red,Small_circle_center[i][j],small_r-2,width=0)
            elif position[i][j] == 2:
                pygame.draw.circle(small_board_move,blue,Small_circle_center[i][j],small_r-2,width=0)
        #Blinking Effect on the small board
            elif red_possible[i][j] == 'Y' and k % 2 == 0 and (next_game == big_cell+1 or next_game == i):
                pygame.draw.circle(blinking,red,Small_circle_center[i][j],small_r-3,width=3)
            elif blue_possible[i][j] == 'Y' and k % 2 == 1 and (next_game == big_cell+1 or next_game == i):
                pygame.draw.circle(blinking,blue,Small_circle_center[i][j],small_r-3,width=3)
            else:
                pygame.draw.circle(blinking,white,Small_circle_center[i][j],small_r-3,width=3)
        
            
    blinking.set_alpha(alpha)   
    pos = (0,0)    

       
    screen.fill((255,255,255))
    screen.blit(board_img,(size[0] // 2 - (window_size[0] // 2),size[1] // 2 - (window_size[1] // 2)))
    screen.blit(big_board_move,(size[0] // 2 - (window_size[0] // 2),size[1] // 2 - (window_size[1] // 2)))
    screen.blit(small_board_move,(size[0] // 2 - (window_size[0] // 2),size[1] // 2 - (window_size[1] // 2)))
    screen.blit(blinking,(size[0] // 2 - (window_size[0] // 2),size[1] // 2 - (window_size[1] // 2)))

        #Start big board winning conditions checks
    if (all(i == 'N' for i in Big_board_possible_blue)) and c == 'P1':
        screen.blit(P1_Win,(size[0] // 2 - (P1_Win_size[0] //2),(size[1] * 1// 3) // 2  - (P1_Win_size[1] //2)))
    elif (all(i == 'N' for i in Big_board_possible_red)) and c == 'P2':
        screen.blit(P2_Win,(size[0] // 2 - (P2_Win_size[0] // 2),(size[1] * 1// 3) // 2  - (P1_Win_size[1] //2)))
    elif red_possible == ref_N and (all(i == ref_N for i in red_possible)) and k % 2 == 0: 
        screen.blit(P2_Win,(size[0] // 2 - (P2_Win_size[0] // 2),(size[1] * 1// 3) // 2  - (P1_Win_size[1] //2)))
        #If player 1 no longer has a move on the small board, then they lose
    elif blue_possible == ref_N and (all(i == ref_N for i in blue_possible)) and k % 2 == 1:
        screen.blit(P1_Win,(size[0] // 2 - (P1_Win_size[0] //2),(size[1] * 1// 3) // 2  - (P1_Win_size[1] //2)))


    '''Another way we can do this is if a player doesn't have a move, then the turn is skipped. '''
    '''
    elif (all(i == ref_N for i in red_possible)) and k % 2 == 0: 
        k = k + 1
    elif (all(i == ref_N for i in blue_possible)) and k % 2 == 1:
        k = k + 1
    '''
    #End big board winning conditions checks 

    
    pygame.display.flip()
    

In [ ]:
pygame.quit()